In [39]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math

In [40]:
# NN_INPUT_DIM = 120
# NN_HIDDEN_DIM = 84
# NN_OUTPUT_DIM = 10
# input_layer_dim = NN_INPUT_DIM
# hidden_layer_dim = NN_HIDDEN_DIM
# output_layer_dim = NN_OUTPUT_DIM

# def create_weight_matrix():
#     weight_input_hidden = np.random.randn(120, 84)
#     weight_hidden_output = np.random.randn(84, 10)

#     return weight_input_hidden, weight_hidden_output

def sigmoid(data):
    return 1 / (1 + np.exp(-data))

# def forward_pass(input): 
#     weight_input_hidden, weight_hidden_output = create_weight_matrix()
#     result_input_hidden = np.dot(input, weight_input_hidden)
#     result_input_hidden_sigmoid = sigmoid(result_input_hidden)

#     result_hidden_output = np.dot(result_input_hidden_sigmoid, weight_hidden_output)
#     result_hidden_output_sigmoid = sigmoid(result_hidden_output)

#     return result_hidden_output_sigmoid

In [47]:
IMAGE_PATH = '/home/aishwarya/CSIS/SMAI/SMAI_assig/a-6/neck_front.jpeg'
# REDUCED_IMAGE_SIZE_ROW = 32
# REDUCED_IMAGE_SIZE_COLUMN = 32
Filter_Row_Dim = 5
Filter_Col_Dim = 5
# FirstLayer_Filters = 6
# SecondLayer_Filters = 16
FC_Convolution_Filter_Count = 120
POOLING_ROW_DIM = 2
POOLING_COL_DIM = 2

def create_image_matrix():
    image = Image.open(IMAGE_PATH)
#     image.show()
    image = image.resize((32,32), Image.ANTIALIAS)
    #image.show() 
    array_image = np.array(image)
#     image.show()
    return array_image

def create_filters(layer_count):
    filter_depth = 0
    if layer_count == 1:
        filter_depth = 6
    elif layer_count == 2:
        filter_depth = 16
    elif layer_count == 3:
        filter_depth = FC_Convolution_Filter_Count
    else:
        print("Invalid layer count")

    filter_matrix = np.random.randn(Filter_Row_Dim, Filter_Col_Dim, filter_depth)
    return filter_matrix

def convolve_oper(data, filter):  
    single_convolve = np.multiply(data, filter)
    return single_convolve

def convolve(data, filter):
    filter_row, filter_col, num_filters = filter.shape
    data_row, data_col, data_channels = data.shape
    conv_result_dim = data_row - filter_row + 1
    ans=0
    convolve_result = np.zeros((conv_result_dim, conv_result_dim, num_filters))

    # Breaking the data into sub-matrices of filter size and calling convolve_oper
    for filter_index in range(num_filters):        
        for x_index in range(conv_result_dim):
            for y_index in range(conv_result_dim):
                ans=convolve_oper(data[x_index: x_index + filter_row, y_index: y_index + filter_col, filter_index % data_channels],filter[:, :, filter_index])
                convolve_result[x_index][y_index][filter_index] = ans.sum()
    return convolve_result

# Implementing max pool operation
# def pool_operation(data):
#     return data.max()

def softmax(w, t=1.0):
    e = np.exp(w / t)
    dist = e / np.sum(e)
    return dist

def ReLU(x):
    return np.where(x > 0, 1.0, 0.0)

def imageshow(x):
    x = x.transpose(2,0,1)
    for i in range(len(x)):
        img = Image.fromarray(np.array(x[i]))
        image = img.resize((200,200),Image.ANTIALIAS)
        image.show()

def imgshow(x):
    #x = x.transpose(2,0,1) 
    img = Image.fromarray(x,'RGB')
    img = img.resize((300,300))
    img.show()      

def max_pooling(data, pool_size_row, pool_size_col):
    data_row, data_col, result_layer = data.shape
    result_row= data_row // pool_size_row
    result_col= data_col // pool_size_col

    pool_result = np.zeros((result_row, result_col, result_layer))

    for layer in range(result_layer):
        for row in range(0, data_row, pool_size_row):
            for col in range(0, data_col, pool_size_col):
                pool_result[row // pool_size_row][col // pool_size_col][layer] = (
                    data[row:row + pool_size_row, col:col + pool_size_col, layer]).max()

    return pool_result

In [52]:
if __name__ == '__main__':
    # Lenet Architectue
    # INPUT => CONV => RELU => POOL => CONV => RELU => POOL => FC(Conv) => RELU => FC

    image = Image.open(IMAGE_PATH)
#     image.show()
    image = image.resize((32,32), Image.ANTIALIAS)
#     im=image.show() 
    image.save('resized_img_32x32.jpeg')
    

# im = ax.imshow(data, cmap=plt.get_cmap('hot'), interpolation='nearest',
#                vmin=0, vmax=1)
# fig.colorbar(im)
# plt.show()
    
    image_array = np.array(image)
    print("Image new Dimensions: ", image_array.shape)

    # -------------------------- First Convolution Block -------------------------------------------
    # Creating first layer filter
    # apply convolution on original image with 6 diffrent filters
    print '\n\nFirst convolution:'
    filter_matrix = create_filters(1)
    print("Filter Dimensions: ", filter_matrix.shape)

    # Convolution at first layer
    convolve_result = convolve(image_array, filter_matrix)
    print("Convolution Result Dimensions:", convolve_result.shape)

    im2=imgshow(convolve_result)
    im2.save('convolve_result.jpeg')

    # Applying ReLU activation function
    relu_result = ReLU(convolve_result)
    print("ReLU Result Dimensions:", relu_result.shape)

#     imgshow(relu_result)

    # MaxPooling at the first convolution block
    pool_result = max_pooling(relu_result, POOLING_ROW_DIM, POOLING_COL_DIM)
    print("Pooling Result Dimensions:", pool_result.shape)

#     imgshow(pool_result)

    # --------------Second Convolution Block-----------------------------------------
    print '\n\nSecond convolution:'
    filter_matrix = create_filters(2)
    print("Filter Dimensions (2nd Block): ", filter_matrix.shape) 

#     imgshow(filter_matrix)

    convolve_result = convolve(pool_result, filter_matrix)
    print("Convolution Result Dimensions (2nd Block):", convolve_result.shape)

#     imgshow(convolve_result)

    # Applying ReLU activation function
    relu_result = ReLU(convolve_result)
    print("ReLU Result Dimensions:", relu_result.shape)

    pool_result = max_pooling(relu_result, POOLING_ROW_DIM, POOLING_COL_DIM)
#     print 'pool result'
#     print pool_result
    print("Pooling Result Dimensions (2nd Block):", pool_result.shape)

    # -------------------- Convolution to give Fully Connected Layer-----------------------
    print '\n\nFully Connected Layer-----------------------'
    
    # -------------------- Convolution to give Fully Connected Layer-----------------------
    print '[forward propagation]\nConvolution to give Fully Connected Layer:-----------'
    
    input_mat=np.array(pool_result.flatten().reshape(1, len(pool_result.flatten())))
    print 'current shape-',input_mat.shape

    # -----------first layer-----------
    weight_mat=np.random.uniform(low=-1, high=1, size=(400,120) )
    output_layer1=np.matmul(input_mat, weight_mat)
    for i in range( len (output_layer1[0]) ):
        output_layer1[0][i]=sigmoid(output_layer1[0][i])

#     print (output_layer1.shape , 'output layer 1')
    print '\noutput layer 1 shape-',output_layer1.shape

    # -----------second layer-----------
    input_mat=output_layer1
    weight_mat=np.random.uniform(low=-1, high=1, size=(120,84) )
    output_layer2=np.matmul(input_mat, weight_mat)
    for i in range( len (output_layer2[0]) ):
        output_layer2[0][i]=sigmoid(output_layer2[0][i])

#     print (output_layer2.shape , 'op layer 2')
    print '\noutput layer 2 shape-',output_layer2.shape

    # -----------third layer-----------
    input_mat=output_layer2
    weight_mat=np.random.uniform(low=-1, high=1, size=(84,10) )
    output_layer3=np.matmul(input_mat, weight_mat)
    for i in range( len (output_layer3[0]) ):
        output_layer3[0][i]=sigmoid(output_layer3[0][i])

    print '\noutput layer 3 shape-',output_layer3.shape
    print '\noutput layer 3 -',output_layer3

('Image new Dimensions: ', (32, 32, 3))


First convolution:
('Filter Dimensions: ', (5, 5, 6))
('Convolution Result Dimensions:', (28, 28, 6))


AttributeError: 'NoneType' object has no attribute 'save'